# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [19]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) Número de valores missings 
df_sinasc = pd.DataFrame(sinasc)
df_sinasc.isnull().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) Então crie uma seleção dessa base somente com as colunas que interessam. São elas: 
#['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 

#primeiro vamos ver os nomes de todas as colunas.

df_sinasc.columns


Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

In [4]:
# 3) Então crie uma seleção dessa base somente com as colunas que interessam. São elas: 
#['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
# Apagar as colunas que não serão usadas
df_sinasc_ajustada = df_sinasc.drop(['ORIGEM','CODESTAB','CODMUNNASC','CODOCUPMAE','QTDFILMORT','CODMUNRES','PARTO','DTNASC',
                                    'HORANASC','SEXO','APGAR1','RACACOR','PESO','IDANOMAL','DTCADASTRO','CODANOMAL','NUMEROLOTE',
                                    'VERSAOSIST','DTRECEBIM','DIFDATA','DTRECORIGA','NATURALMAE','CODMUNNATU','CODUFNATU','ESCMAE2010',
                                    'SERIESCMAE','DTNASCMAE','RACACORMAE','QTDGESTANT','QTDPARTNOR','QTDPARTCES','IDADEPAI','DTULTMENST',
                                    'SEMAGESTAC','TPMETESTIM','CONSPRENAT','MESPRENAT','TPAPRESENT','STTRABPART','STCESPARTO','TPNASCASSI',
                                    'TPFUNCRESP','TPDOCRESP','DTDECLARAC','ESCMAEAGR1','STDNEPIDEM','STDNNOVA','CODPAISRES','TPROBSON',
                                    'PARIDADE','KOTELCHUCK','CONTADOR','munResStatus','munResTipo','munResNome','munResUf','munResLat',
                                    'munResLon','munResAlt','munResArea'], axis=1)
df_sinasc_ajustada

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [5]:
# 3) Contando os valores missing da nova df
df_sinasc_ajustada.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [6]:
# 3) Contando os valores missing da nova df em porcentual
df_sinasc_ajustada.isnull().sum()/27028

#Vemos que a maior quantidade de missing não chega a 6%

LOCNASC       0.000000
IDADEMAE      0.000000
ESTCIVMAE     0.011729
ESCMAE        0.011544
QTDFILVIVO    0.058199
GESTACAO      0.045582
GRAVIDEZ      0.002923
CONSULTAS     0.000000
APGAR5        0.003811
dtype: float64

In [7]:
# 4) Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido.
df_sinasc_ajustada.APGAR5.isnull().sum()

103

In [8]:
# 4) Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido.

df_sinasc_ajustada.dropna(subset = ['APGAR5'], inplace = True)
df_sinasc_ajustada.APGAR5.isnull().sum()

0

In [9]:
# 5) as variáveis ['ESTCIVMAE', 'CONSULTAS'] possuem o código 9, que significa ignorado. Vamos assumir que o não preenchido é o mesmo que o código 9
# assim sendo, substitui os Nan de 'ESTCIVMAE' por 9, para consultas não foi necessários pois não temos Nan
df_sinasc_ajustada['ESTCIVMAE'].replace(np.nan,(9),inplace = True)
df_sinasc_ajustada

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [10]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero
df_sinasc_ajustada['QTDFILVIVO'].replace(np.nan,(0),inplace = True)
# verificando se existe dados Nan em (QTDFILVIVO)
df_sinasc_ajustada.QTDFILVIVO.isnull().sum()

0

In [11]:
# 7) Verificando e tratando os Nan restantes 
df_sinasc_ajustada.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [12]:
# 7) ainda temos 'ESCMAE', 'GESTACAO' e 'GRAVIDEZ' com dados missing.
# todos estes dados são textos, que representam um valor aproximado, sendo assim, vamos substituir os missings pelo item com maior quantidade.

# verificando as quantidades de 'ESCMAE'
df_sinasc_ajustada.groupby(['ESCMAE']).size()
# '8 a 11 anos' foi o tema com maior quantidade
df_sinasc_ajustada['ESCMAE'].replace(np.nan,('8 a 11 anos'),inplace = True)

# verificando as quantidades de 'GESTACAO'
df_sinasc_ajustada.groupby(['GESTACAO']).size()
# '37 a 41 semanas' foi o tema com maior quantidade - que também é o tempo mais comum e dado com natural
df_sinasc_ajustada['GESTACAO'].replace(np.nan,('37 a 41 semanas'),inplace = True)

# verificando as quantidades de 'GRAVIDEZ'
df_sinasc_ajustada.groupby(['GRAVIDEZ']).size()
# 'Única' foi o tema com maior quantidade - que também é o tempo mais comum.
df_sinasc_ajustada['GRAVIDEZ'].replace(np.nan,('Única'),inplace = True)

#base limpa sem nenhum missing
df_sinasc_ajustada.isnull().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [13]:
# 8) Classificações do APGAR5
# Entre 8 e 10 está em uma faixa 'normal'.
# Entre 6 e 7, 'asfixia leve'.
# Entre 4 e 5 'asfixia moderada'.
# Entre 0 e 3 'asfixia severa'.

df_sinasc_ajustada.loc[df_sinasc_ajustada['APGAR5']>=8, 'CATAPGAR5'] = 'normal'
df_sinasc_ajustada.loc[(df_sinasc_ajustada['APGAR5']>=6) & (df_sinasc_ajustada['APGAR5']<=7), 'CATAPGAR5'] = 'asfixia leve'
df_sinasc_ajustada.loc[(df_sinasc_ajustada['APGAR5']>=4) & (df_sinasc_ajustada['APGAR5']<=5), 'CATAPGAR5'] = 'asfixia moderada'
df_sinasc_ajustada.loc[df_sinasc_ajustada['APGAR5']<=3, 'CATAPGAR5'] = 'asfixia severa'

df_sinasc_ajustada

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CATAPGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


In [18]:
# 8) calcule a frequência da categorização criada 'CATAPGAR5'
df_sinasc_ajustada.groupby(['CATAPGAR5']).size()/26926


CATAPGAR5
asfixia leve        0.011884
asfixia moderada    0.002525
asfixia severa      0.002748
normal              0.982805
dtype: float64

In [41]:
# 9) Renomeie as variáveis para que fiquem no snake case
# colunas para fazermos o ajuste: ['ESCMAE'], ['GESTACAO'], ['GRAVIDEZ'] e ['CATAPGAR5']

# verificando as quantidades de 'ESCMAE'
df_sinasc_ajustada.groupby(['ESCMAE']).size()
# '1 a 3 anos', '12 anos ou mais', '4 a 7 anos', '8 a 11 anos', 'Nenhuma'
df_sinasc_ajustada['ESCMAE'].replace(['1 a 3 anos'],'1_a_3_anos',inplace = True)
df_sinasc_ajustada['ESCMAE'].replace(['12 anos ou mais'],'12_anos_ou_mais',inplace = True)
df_sinasc_ajustada['ESCMAE'].replace(['4 a 7 anos'],'4_a_7_anos',inplace = True)
df_sinasc_ajustada['ESCMAE'].replace(['8 a 11 anos'],'8_a_11_anos',inplace = True)
df_sinasc_ajustada['ESCMAE'].replace(['Nenhuma'],'nenhuma',inplace = True)

# verificando as quantidades de 'GESTACAO'
df_sinasc_ajustada.groupby(['GESTACAO']).size()
#'22 a 27 semanas', '28 a 31 semanas', '32 a 36 semanas', '37 a 41 semanas', '42 semanas e mais', 'Menos de 22 semanas'
df_sinasc_ajustada['GESTACAO'].replace(['22 a 27 semanas'],'22_a_27_semanas',inplace = True)
df_sinasc_ajustada['GESTACAO'].replace(['28 a 31 semanas'],'28_a_31_semanas',inplace = True)
df_sinasc_ajustada['GESTACAO'].replace(['32 a 36 semanas'],'32_a_36_semanas',inplace = True)
df_sinasc_ajustada['GESTACAO'].replace(['37 a 41 semanas'],'37_a_41_semanas',inplace = True)
df_sinasc_ajustada['GESTACAO'].replace(['42 semanas e mais'],'42_semanas_e_mais',inplace = True)
df_sinasc_ajustada['GESTACAO'].replace(['Menos de 22 semanas'],'Menos_de_22_semanas',inplace = True)

# verificando as quantidades de 'GRAVIDEZ'
df_sinasc_ajustada.groupby(['GRAVIDEZ']).size()
#'Dupla', 'Tríplice e mais', 'Única'
df_sinasc_ajustada['GRAVIDEZ'].replace(['Dupla'],'dupla',inplace = True)
df_sinasc_ajustada['GRAVIDEZ'].replace(['Tríplice e mais'],'tríplice_e_mais',inplace = True)
df_sinasc_ajustada['GRAVIDEZ'].replace(['Única'],'única',inplace = True)

# verificando as quantidades de 'CATAPGAR5'
df_sinasc_ajustada.groupby(['CATAPGAR5']).size()
#'asfixia leve', 'asfixia moderada', 'asfixia severa'
df_sinasc_ajustada['CATAPGAR5'].replace(['asfixia leve'],'asfixia_leve',inplace = True)
df_sinasc_ajustada['CATAPGAR5'].replace(['asfixia moderada'],'asfixia_moderada',inplace = True)
df_sinasc_ajustada['CATAPGAR5'].replace(['asfixia severa'],'asfixia_severa',inplace = True)

df_sinasc_ajustada

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,CATAPGAR5
0,1,19,5.0,8_a_11_anos,0.0,37_a_41_semanas,única,4,10.0,normal
1,1,29,2.0,8_a_11_anos,1.0,37_a_41_semanas,única,4,9.0,normal
2,1,37,9.0,8_a_11_anos,2.0,37_a_41_semanas,única,4,10.0,normal
3,1,30,5.0,12_anos_ou_mais,0.0,37_a_41_semanas,única,3,10.0,normal
4,1,30,2.0,8_a_11_anos,1.0,37_a_41_semanas,única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12_anos_ou_mais,1.0,32_a_36_semanas,única,4,9.0,normal
27024,1,19,1.0,8_a_11_anos,0.0,37_a_41_semanas,única,4,9.0,normal
27025,1,24,2.0,8_a_11_anos,0.0,37_a_41_semanas,única,4,10.0,normal
27026,1,21,2.0,8_a_11_anos,1.0,32_a_36_semanas,única,4,9.0,normal
